# AWS Resources deployment using a Jupyter Notebook

In [1]:
# !python3 -m pip install boto3
# !python3 -m pip install requests
# !python3 -m pip install tqdm
# !python3 -m pip install pandas
# !python3 -m pip install s3fs
# !python3 -m pip install -q -U paramiko
# !python3 -m pip install -q -U scp

In [2]:
import IPython
import boto3
import time
import os
import json
import requests
import pandas as pd
import paramiko
import scp
from zipfile import ZipFile
from urllib.request import urlopen
from tqdm.notebook import tqdm

#### Some basic settings

In [3]:
my_bucket_name = 'dantohe-my-experimental-iac-01'
my_region = 'us-west-2'
stem = 'my-experimental'
my_InstanceProfileName = f'{stem}-InstanceProfileName-iac-01'
ec2_pem_name      = f'{stem}-kp-june-2021-01'
my_role_name = f'{stem}-ec2-role-01'
my_security_group_name = f'{stem}-Airflow-security-group-01'

ec2 = boto3.client('ec2')
iam = boto3.client('iam')

#### Create an EC2 key-pair    
If the key already exists then don't do anything.    

In [4]:

key_exists = False

response = ec2.describe_key_pairs()['KeyPairs']
for key in response:
    if key['KeyName'] == ec2_pem_name:
        key_exists = True
    found_instance = ec2.describe_instances(
        Filters=[
            {
                'Name': 'key-name',
                'Values': [key['KeyName']]
            }
        ]
    )['Reservations']
#     if len(found_instance) == 0:
#         print (key['KeyName'] + " is unused")

if key_exists:
    print('key already exists')
else:
    ec2_pem_path = f'./{ec2_pem_name}.pem'
    if os.path.isfile(ec2_pem_path):
        os.remove(ec2_pem_path)
    ec2_keypair = ec2.create_key_pair(KeyName=ec2_pem_name)
    with open(ec2_pem_path, 'w+') as ec2_pem_file:
        ec2_pem_file.write(str(ec2_keypair['KeyMaterial']))
    !chmod 400 {ec2_pem_path}
    print(f'{ec2_pem_name} has been created sucessfully and the pem is available at\n{ec2_pem_path}')
    

key already exists


#### EC2 Resources    
IAM - refrences: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/iam.html#role   


In [5]:
roles = iam.list_roles()
role_list = roles['Roles']
ec2_role= None

for key in role_list:
    if key['RoleName'] == my_role_name:
        ec2_role = key

if ec2_role is not None:
    print(f'Role {my_role_name} already exists')
else:
    ec2_role = iam.create_role(
        Path='/',
        RoleName=my_role_name,
        Description='',
        MaxSessionDuration=3600,
        AssumeRolePolicyDocument="""{
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": { "Service": "ec2.amazonaws.com"},
          "Action": "sts:AssumeRole"
        }
      ]
    }""".replace('<dw_bucket>', my_bucket_name))['Role']
    ### Also atach the S3 policy to the role
    for ec2_policy in  [
        'arn:aws:iam::aws:policy/AmazonS3FullAccess']:
        assert iam.attach_role_policy(
            RoleName=ec2_role['RoleName'],
            PolicyArn=ec2_policy)['ResponseMetadata']['HTTPStatusCode'] == 200
    
    print(f'{my_role_name} has been createed - the S3 policies has been attached')
#     ec2_role['Arn']

my-experimental-ec2-role-01 has been createed - the S3 policies has been attached


#### Attaches S3 acces policy to the role

In [6]:
# for ec2_policy in  [
#         'arn:aws:iam::aws:policy/AmazonS3FullAccess']:
#     assert iam.attach_role_policy(
#         RoleName=ec2_role['RoleName'],
#         PolicyArn=ec2_policy)['ResponseMetadata']['HTTPStatusCode'] == 200

#### Creates the instance profile AND adds the role to instance profile

In [6]:
instance_profiles = iam.list_instance_profiles()
instance_profiles_list = instance_profiles['InstanceProfiles']
ec2_instance_profile = None

# existing_instance_profile_names =[]

for key in instance_profiles_list:
    if key['InstanceProfileName'] == my_InstanceProfileName:
        ec2_instance_profile =key
#     existing_instance_profile_names.append(key['InstanceProfileName'])

# if my_InstanceProfileName in existing_instance_profile_names:
#     print(f'{my_InstanceProfileName} already exists')
if ec2_instance_profile is not None:
    print(f'{my_InstanceProfileName} already exists')
else:
    #creates the instaance profile
    ec2_instance_profile = iam.create_instance_profile(InstanceProfileName=my_InstanceProfileName)['InstanceProfile']
    iam.get_waiter('instance_profile_exists').wait(InstanceProfileName=my_InstanceProfileName)

    #adds the role to the instance profile
    assert iam.add_role_to_instance_profile(InstanceProfileName=ec2_instance_profile['InstanceProfileName'], RoleName=ec2_role['RoleName'])['ResponseMetadata']['HTTPStatusCode'] == 200
    print(f'{my_InstanceProfileName} has been created')

my-experimental-InstanceProfileName-iac-01 has been created


#### Createting a security group

In [7]:
security_groups = ec2.describe_security_groups()
existing_security_groups = security_groups['SecurityGroups']

ec2_sg = None

for key in existing_security_groups:
    if key['GroupName'] == my_security_group_name:
      ec2_sg=key  
    
if ec2_sg is not None:
    print(f'The security group {my_security_group_name} already exists')
else:
    ec2_sg = ec2.create_security_group(
        Description='Allows 22 trafic',
        GroupName=my_security_group_name)
    ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=22, ToPort=22, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
    ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=8080, ToPort=8080, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
    ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=5555, ToPort=5555, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
    ec2.authorize_security_group_ingress(CidrIp='0.0.0.0/0', FromPort=3306, ToPort=3306, GroupId=ec2_sg['GroupId'], IpProtocol='TCP')
    print(f'The security group {my_security_group_name} has been created')

The security group my-experimental-Airflow-security-group-01 has been created


#### Requesting spot instance(s)

In [8]:
# time.sleep(30) #wait instance profile...
# ec2_ami_id = 'ami-0aeeebd8d2ab47354'
#Amazon Linux AMI - it has some issues and complications with installing mysql and airflow
# ec2_ami_id = 'ami-0aeeebd8d2ab47354'
#defaulting to ubuntu
ec2_ami_id = 'ami-09e67e426f25ce0d7'
ec2_spot = ec2.request_spot_instances(
    AvailabilityZoneGroup='us-east-1',
    InstanceCount=1,
    LaunchSpecification={
        'SecurityGroupIds': [ec2_sg['GroupId']],
        'EbsOptimized': False,
        'KeyName': ec2_pem_name,
        'ImageId': ec2_ami_id,
        'InstanceType': 't3.large',
        'IamInstanceProfile': {
            'Arn': ec2_instance_profile['Arn']
        },
        "BlockDeviceMappings": [
            {
                "DeviceName": "/dev/sda1",
                "Ebs": {
                        "DeleteOnTermination": True,
                        "VolumeSize": 30,
                        "Encrypted": False,
                        "VolumeType": "gp2"
                }
            }
        ],
    },
    SpotPrice='0.10',
    Type='one-time',
    InstanceInterruptionBehavior='terminate'
)
ec2_spot_id = ec2_spot['SpotInstanceRequests'][0]['SpotInstanceRequestId']
ec2.get_waiter('spot_instance_request_fulfilled').wait(SpotInstanceRequestIds=[ec2_spot_id])
ec2_spot_id

'sir-yv6hjdwg'

## Clean Up

#### Terminate spot instance(s)

In [9]:

response = ec2.describe_spot_instance_requests(SpotInstanceRequestIds=[ec2_spot_id])
instances = []
for instance in response['SpotInstanceRequests']:
    instances.append(instance['InstanceId'])
    print(instance['InstanceId'])

try:
    response = ec2.terminate_instances(InstanceIds=instances)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The ec2 instance {ec2_spot_id} has been removed\n\t{response}')
except Exception:
    print(f'The ec2 instance {ec2_spot_id} has NOT been removed\n\t{response} - probably because it did not exist')
    pass  

i-01427e7deb0cbc3f9
The ec2 instance sir-yv6hjdwg has been removed
	{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-01427e7deb0cbc3f9', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'e8535aea-0300-4ee1-ac94-0c783f7d9323', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e8535aea-0300-4ee1-ac94-0c783f7d9323', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'accept-encoding', 'date': 'Fri, 25 Jun 2021 01:01:01 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


#### Removes the security group

In [ ]:
# waiting some time as the instance might take a moment to shutdown
# time.sleep(60)
ec2.delete_security_group(GroupName=my_security_group_name)['ResponseMetadata']['HTTPStatusCode']
try:
    if ec2.delete_security_group(GroupName=my_security_group_name)['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'{my_security_group_name} has been deleted')
except Exception:
    print(f'{my_security_group_name} has NOT BEEN deleted - most likely it did not exist')
    pass  

#### Removes role from instance profile

In [14]:
# removes role from instance profile
try:
    response = iam.remove_role_from_instance_profile(
        InstanceProfileName=my_InstanceProfileName,
        RoleName=my_role_name
    )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The role {my_role_name} has been removed from {my_InstanceProfileName} \n\t{response}')
except Exception:
    print(f'The role {my_role_name} has been NOT BEEEN removed from {my_InstanceProfileName} - most likely it was not attached in the first place')
    pass  


The role my-experimental-ec2-role-01 has been removed from my-experimental-InstanceProfileName-iac-01 
	{'ResponseMetadata': {'RequestId': '572059ef-f0eb-4756-96d9-4c28bfcdc837', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '572059ef-f0eb-4756-96d9-4c28bfcdc837', 'content-type': 'text/xml', 'content-length': '238', 'date': 'Fri, 25 Jun 2021 01:05:14 GMT'}, 'RetryAttempts': 0}}


#### Removes the instance profile 

In [15]:
try:
    response = iam.delete_instance_profile(InstanceProfileName=my_InstanceProfileName)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The instance profile  {my_InstanceProfileName} has been removed \n\t{response}')
except Exception:
    print(f'The instance profile  {my_InstanceProfileName} has NOT BEEN removed most likely it did not exist in the first place')
    pass  

The instance profile  my-experimental-InstanceProfileName-iac-01 has been removed 
	{'ResponseMetadata': {'RequestId': '64987f20-98fb-4551-b767-115a83002404', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '64987f20-98fb-4551-b767-115a83002404', 'content-type': 'text/xml', 'content-length': '222', 'date': 'Fri, 25 Jun 2021 01:05:22 GMT'}, 'RetryAttempts': 0}}


#### Detach the policies from the role

In [16]:
for ec2_policy in  [
        'arn:aws:iam::aws:policy/AmazonS3FullAccess']:
    assert iam.detach_role_policy(
        RoleName=ec2_role['RoleName'],
        PolicyArn=ec2_policy)['ResponseMetadata']['HTTPStatusCode'] == 200
# response = client.detach_role_policy(
#     RoleName='string',
#     PolicyArn='string'
# )    

#### Removes the role

In [17]:
try:
    response = iam.delete_role(RoleName=my_role_name)
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print(f'The role  {my_InstanceProfileName} has been removed \n\t{response}')
except Exception:
    print(f'The resource {my_role_name} has NOT BEEN removed most likely it did not exist in the first place')
    pass  



The role  my-experimental-InstanceProfileName-iac-01 has been removed 
	{'ResponseMetadata': {'RequestId': '0cbb3ea3-0e7f-4685-b99d-ff92df367578', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0cbb3ea3-0e7f-4685-b99d-ff92df367578', 'content-type': 'text/xml', 'content-length': '200', 'date': 'Fri, 25 Jun 2021 01:05:29 GMT'}, 'RetryAttempts': 0}}
